# Introduction

This is an analysis of data from the National Hockey League. This was done for Foundations of Data Science in Fall of 2019. This project was done by Group 5:
- Andokie Ibeshi
- Daniel Basilio
- Prabuddh Dixit
- Vineeta Kuckreja


## Data Notes
All of the data in this analysis was fetched from Natural Stat Trick. We used individal player stats for the last five years, regular season data only. Each of the CSVs contains all players who registered any game time throughout that year. We got the data as individual year CSV's for a few reasons:
- The data was a lot easier to download in chunks
- The data does not have a year label, so by importing them separately, we can attach year labels from the filenames

### A note on years
Hockey seasons span two calendar years because the season starts in October each year, and continues until the playoffs begin, typically in April. There have been events which shift the start/end dates (the World Cup of Hockey in 2016, the Olympics in 2014) but October -> April should apply most years. Each row of data would need to be marked with two years (i.e. 2014-2015) to represent the two calendar years the season was played over. To keep the year data simpler to understand, we will mark all the data as the year that the Stanley Cup was awared. For example, 2014-2015 will be marked as simply 2015, as that seasons Stanley Cup was awarded at the end of the season in 2015.

In [62]:
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [63]:
def map_positions(position):
    if 'C' in position or 'R' in position or 'L' in position:
        return 'F'
    else:
        return position

In [64]:
def read_player_data(filename):
    data = pd.read_csv(
        filename,
        usecols=[
            "Player",
            "Team",
            "Position",
            "GP",
            "Goals",
            "Total Assists",
            "First Assists",
            "Second Assists",
            "Total Points",
            "iCF",
            "Shots",
            "SH%",
            "Hits",
            "Hits Taken",
        ]
    )
    return data

In [76]:
def clean_player_data(data, year):
    # Remove all players who failed to register at least 20 games played.
    data = data.drop(data[data["GP"] < 20].index)
    
    # Cast the SH% column to float (it imports as object for some reason)
    data = data.astype({"SH%": "float64"})
    
    # Attach the year as a column
    data["Year"] = year
    
    # Rename iCF to Shot Attempts to be clearer
    data = data.rename(columns={"iCF": "Shot Attempts"})
    
    # Map all forward positions to just 'F' to make the data nicer to work with
    data["Position"] = data["Position"].apply(map_positions)
    
    # Set the index to be a combo of player and year
    data = data.set_index(["Year", "Player"])
    
    return data

In [66]:
# Read and clean all of our data
def read_and_clean_player_data(filename, year):
    year_data = read_player_data(filename)
    cleaned_year_data = clean_player_data(year_data, year)
    return cleaned_year_data

In [67]:
# Read and combine all of Data Frames together
def fetch_player_data():
    fourteen_fifteen = read_and_clean_player_data('./yearly-player-data/14-15.csv', '2015')
    fifteen_sixteen = read_and_clean_player_data('./yearly-player-data/15-16.csv', '2016')
    sixteen_seventeen = read_and_clean_player_data('./yearly-player-data/16-17.csv', '2017')
    seventeen_eighteen = read_and_clean_player_data('./yearly-player-data/17-18.csv', '2018')
    eighteen_nineteen = read_and_clean_player_data('./yearly-player-data/18-19.csv', '2019')
    player_data = fourteen_fifteen.append([
        fifteen_sixteen,
        sixteen_seventeen,
        seventeen_eighteen,
        eighteen_nineteen,
    ])
    return player_data

In [68]:
fetch_player_data()

Team Position  GP  Goals  Total Assists  \
Year Player                                                       
2015 Jamie Benn          DAL        F  82     35             52   
     John Tavares        NYI        F  82     38             48   
     Sidney Crosby       PIT        F  77     28             56   
     Alex Ovechkin       WSH        F  81     53             28   
     Jakub Voracek       PHI        F  82     22             59   
     Nicklas Backstrom   WSH        F  82     18             60   
     Tyler Seguin        DAL        F  71     37             40   
     Daniel Sedin        VAN        F  82     20             56   
     Jiri Hudler         CGY        F  78     31             45   
     Henrik Sedin        VAN        F  82     18             55   
     Nick Foligno        CBJ        F  79     31             42   
     Claude Giroux       PHI        F  81     25             48   
     Vladimir Tarasenko  STL        F  77     37             36   
     Steven Stamkos      T.B        F  82     43             29   
     Tyler Johnson       T.B        F  77     29             43   
     Ryan Johansen       CBJ        F  82     26             45   
     Ryan Getzlaf        ANA        F  77     25             45   
     Joe Pavelski        S.J        F  82     37             33   
     Evgeni Malkin       PIT        F  69     28             42   
     Rick Nash           NYR        F  79     42             27   
     Logan Couture       S.J        F  82     27             40   
     Max Pacioretty      MTL        F  80     37             30   
     Henrik Zetterberg   DET        F  77     17             49   
     Jonathan Toews      CHI        F  81     28             38   
     Erik Karlsson       OTT        D  82     21             45   
     Joe Thornton        S.J        F  78     16             49   
     Pavel Datsyuk       DET        F  63     26             39   
     Nikita Kucherov     T.B        F  82     29             36   
     Alexander Steen     STL        F  74     24             40   
     Anze Kopitar        L.A        F  79     16             48   
...                      ...      ...  ..    ...            ...   
2019 Ilya Lyubushkin     ARI        D  41      0              4   
     Micheal Haley       FLA        F  24      1              2   
     Jori Lehtera        PHI        F  27      1              2   
     Zac Rinaldo         NSH        F  23      1              2   
     Brandon Manning     CHI        D  27      1              2   
     Ryan Spooner        EDM        F  25      2              1   
     Kevin Gravel        EDM        D  36      0              3   
     Victor Rask         MIN        F  23      2              1   
     Colby Cave          EDM        F  33      2              1   
     Clark Bishop        CAR        F  20      1              2   
     Anders Bjork        BOS        F  20      1              2   
     Michael McLeod      N.J        F  21      0              3   
     Vinni Lettieri      NYR        F  27      1              2   
     Juuso Valimaki      CGY        D  24      1              2   
     Oscar Fantenberg    L.A        D  46      2              1   
     Matt Hendricks      MIN        F  22      0              2   
     Tom Pyatt           OTT        F  37      0              2   
     Ben Lovejoy         DAL        D  20      0              2   
     Luke Witkowski      DET        F  34      0              2   
     Dalton Prout        CGY        D  20      1              1   
     Jordan Schmaltz     STL        D  20      0              2   
     Josh Brown          FLA        D  37      1              1   
     Nick Paul           OTT        F  20      1              1   
     Christian Folin     PHI        D  26      0              2   
     Philippe Myers      PHI        D  21      1              1   
     Cody McLeod         NYR        F  31      1              0   
     Alex Petrovic       FLA        D  26      0              1   
    

In [69]:
def read_team_data(filename):
    data = pd.read_csv(
        filename,
        usecols=[
            "Team",
            "W",
            "L",
            "OTL",
            "Points",
        ]
    )
    return data

In [77]:
def clean_team_data(data, year):
    # Rename a few columns to make them easier to understand 
    data = data.rename(columns={"W": "Wins", "L": "Losses", "OTL": "Overtime Losses"})
    
    # Attach the year as a column
    data["Year"] = year
    
    # Set the index to be a combo of team and year
    data = data.set_index(["Year", "Team"])
    
    return data 

In [71]:
def read_and_clean_team_data(filename, year):
    year_data = read_team_data(filename)
    cleaned_year_data = clean_team_data(year_data, year)
    return cleaned_year_data

In [78]:
def fetch_team_data():
    fourteen_fifteen = read_and_clean_team_data('./yearly-team-data/14-15.csv', '2015')
    fifteen_sixteen = read_and_clean_team_data('./yearly-team-data/15-16.csv', '2016')
    sixteen_seventeen = read_and_clean_team_data('./yearly-team-data/16-17.csv', '2017')
    seventeen_eighteen = read_and_clean_team_data('./yearly-team-data/17-18.csv', '2018')
    eighteen_nineteen = read_and_clean_team_data('./yearly-team-data/18-19.csv', '2019')
    team_data = fourteen_fifteen.append([
        fifteen_sixteen,
        sixteen_seventeen,
        seventeen_eighteen,
        eighteen_nineteen,
    ])
    return team_data

In [79]:
fetch_team_data()

Wins  Losses  Overtime Losses  Points
Year Team                                                        
2015 Anaheim Ducks            51      24                7     109
     Arizona Coyotes          24      50                8      56
     Boston Bruins            41      27               14      96
     Buffalo Sabres           23      51                8      54
     Carolina Hurricanes      30      41               11      71
     Columbus Blue Jackets    42      35                5      89
     Calgary Flames           45      30                7      97
     Chicago Blackhawks       48      28                6     102
     Colorado Avalanche       39      31               12      90
     Dallas Stars             41      31               10      92
     Detroit Red Wings        43      25               14     100
     Edmonton Oilers          24      44               14      62
     Florida Panthers         38      29               15      91
     Los Angeles Kings        40      27               15      95
     Minnesota Wild           46      28                8     100
     Montreal Canadiens       50      22               10     110
     New Jersey Devils        32      36               14      78
     Nashville Predators      47      25               10     104
     New York Islanders       47      28                7     101
     New York Rangers         53      22                7     113
     Ottawa Senators          43      26               13      99
     Philadelphia Flyers      33      31               18      84
     Pittsburgh Penguins      43      27               12      98
     San Jose Sharks          40      33                9      89
     St Louis Blues           51      24                7     109
     Tampa Bay Lightning      50      24                8     108
     Toronto Maple Leafs      30      44                8      68
     Vancouver Canucks        48      29                5     101
     Winnipeg Jets            43      26               13      99
     Washington Capitals      45      26               11     101
...                          ...     ...              ...     ...
2019 Arizona Coyotes          39      35                8      86
     Boston Bruins            49      24                9     107
     Buffalo Sabres           33      39               10      76
     Carolina Hurricanes      46      29                7      99
     Columbus Blue Jackets    47      31                4      98
     Calgary Flames           50      25                7     107
     Chicago Blackhawks       36      34               12      84
     Colorado Avalanche       38      30               14      90
     Dallas Stars             43      32                7      93
     Detroit Red Wings        32      40               10      74
     Edmonton Oilers          35      38                9      79
     Florida Panthers         36      32               14      86
     Los Angeles Kings        31      42                9      71
     Minnesota Wild           37      36                9      83
     Montreal Canadiens       44      30                8      96
     New Jersey Devils        31      41               10      72
     Nashville Predators      47      29                6     100
     New York Islanders       48      27                7     103
     New York Rangers         32      36               14      78
     Ottawa Senators          29      47                6      64
     Philadelphia Flyers      37      37                8      82
     Pittsburgh Penguins      44      26               12     100
     San Jose Sharks          46      27                9     101
     St Louis Blues           45      28                9      99
     Tampa Bay Lightning      62      16                4     128
     Toronto Maple Leafs      46      28                8     100
     Vancouver Canucks        35      36               11      81
     Vegas Golden Knights     43      32                7      93
  